In [ ]:
import sys
sys.path.append(r"C:\Users\Talha Ali\Desktop\machine learning labs\timeseires")

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [7]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = create_lstm()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 24, 8)               │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 20)                  │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,301 (12.89 KB)

 Trainable params: 3,301 (12.89 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Talha Ali\Desktop\machine learning labs'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\PMLS\AppData\Local\Temp\ipykernel_9008\292278908.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\PMLS\AppData\Local\Temp\ipykernel_9008\292278908.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [15]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [17]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [ ]:
import os
path_dataset =r'C:\Users\Talha Ali\Desktop\machine learning labs)\lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [21]:
time_steps=24
num_features=21

In [23]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.3705260753631592 sec


In [27]:
epochs = 20
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/20
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0081 - mae: 0.0081 - mape: 846.5244
Epoch 1: val_loss did not improve from 0.00805
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - loss: 0.0081 - mae: 0.0081 - mape: 845.4700 - val_loss: 0.0090 - val_mae: 0.0090 - val_mape: 3.6864
Epoch 2/20
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0079 - mae: 0.0079 - mape: 267.2697
Epoch 2: val_loss improved from 0.00805 to 0.00765, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 0.0079 - mae: 0.0079 - mape: 267.0775 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.1558
Epoch 3/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0076 - mae: 0.0076 - mape: 40.8932
Epoch 3: val_loss did not improve from 0.00765
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 0.0076 - mae: 0.0076 - mape: 41.0155 - val_loss: 0.0080 - val_mae: 0.0080 - val_mape: 3.6880
Epoch 4/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0075 - mae: 0.0075 - mape: 132.1955
Epoch 4: val_loss did not improve from 0.00765
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 0.0075 - mae: 0.0075 - mape: 132.2478 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.3800
Epoch 5/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0074 - mae: 0.0074 - mape: 148.4264
Epoch 5: val_loss improved from 0.00765 to 0.00745, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.0074 - mae: 0.0074 - mape: 148.4783 - val_loss: 0.0074 - val_mae: 0.0074 - val_mape: 3.3070
Epoch 6/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0073 - mae: 0.0073 - mape: 116.5752
Epoch 6: val_loss did not improve from 0.00745
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 0.0073 - mae: 0.0073 - mape: 116.5899 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.3719
Epoch 7/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0071 - mae: 0.0071 - mape: 4.2034
Epoch 7: val_loss improved from 0.00745 to 0.00734, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 0.0071 - mae: 0.0071 - mape: 4.2516 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.3545
Epoch 8/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0071 - mae: 0.0071 - mape: 39.7457
Epoch 8: val_loss improved from 0.00734 to 0.00660, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 12ms/step - loss: 0.0071 - mae: 0.0071 - mape: 39.7328 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 2.7097
Epoch 9/20
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0071 - mae: 0.0071 - mape: 45.3842
Epoch 9: val_loss did not improve from 0.00660
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0071 - mae: 0.0071 - mape: 45.6465 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 3.7781
Epoch 10/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0070 - mae: 0.0070 - mape: 42.2878
Epoch 10: val_loss did not improve from 0.00660
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - loss: 0.0070 - mae: 0.0070 - mape: 42.3944 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.2903
Epoch 11/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0070 - mae: 0.0070 - mape: 88.7448
Epoch 11: val_loss did not improve from 0.00660
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 0.0070 - mae: 0.0070 - mape: 88.8011 - val_loss: 0.0067 - val_ma

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.0069 - mae: 0.0069 - mape: 214.1714 - val_loss: 0.0063 - val_mae: 0.0063 - val_mape: 2.6296
Epoch 14/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0068 - mae: 0.0068 - mape: 419.0279
Epoch 14: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - loss: 0.0068 - mae: 0.0068 - mape: 418.6875 - val_loss: 0.0074 - val_mae: 0.0074 - val_mape: 2.9761
Epoch 15/20
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0067 - mae: 0.0067 - mape: 46.8017
Epoch 15: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 38s 14ms/step - loss: 0.0067 - mae: 0.0067 - mape: 46.8143 - val_loss: 0.0068 - val_mae: 0.0068 - val_mape: 2.9668
Epoch 16/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0068 - mae: 0.0068 - mape: 69.8295
Epoch 16: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 0.0068 - mae: 0.0068 - mape: 69.8106 - val_loss: 0.0064 - v

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0067 - mae: 0.0067 - mape: 27.9094 - val_loss: 0.0063 - val_mae: 0.0063 - val_mape: 2.7222
Epoch 19/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0067 - mae: 0.0067 - mape: 6.9010
Epoch 19: val_loss did not improve from 0.00627
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0067 - mae: 0.0067 - mape: 6.9335 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 2.6801
Epoch 20/20
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0066 - mae: 0.0066 - mape: 9.1326
Epoch 20: val_loss did not improve from 0.00627
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0066 - mae: 0.0066 - mape: 9.1689 - val_loss: 0.0069 - val_mae: 0.0069 - val_mape: 3.1573


In [ ]:

model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\E1-cp-0018-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Mean Absolute Error (MAE): 101.76
Median Absolute Error (MedAE): 80.07
Mean Squared Error (MSE): 18319.45
Root Mean Squared Error (RMSE): 135.35
Mean Absolute Percentage Error (MAPE): 0.69 %
Median Absolute Percentage Error (MDAPE): 0.55 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Talha Ali\Desktop\machine learning labs\E1-cp-0018-loss0.01.h5'
start_epoch= 21

In [ ]:
import os
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, InputLayer
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

# Define your model builder function (replace PC.build)
def build_model(time_steps, num_features, reg=0.0005):
    model = Sequential()
    model.add(InputLayer(input_shape=(time_steps, num_features)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(1))  # Adjust output layer size according to your task
    return model


# Path to save/load your model checkpoint
model_path = r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'

# Construct the callback to save only the *best* model based on validation loss
EpochCheckpoint1 = ModelCheckpoint(
    model_path,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]

# Variable 'model' for holding your model object
model = None

# Check if checkpoint file exists
if not os.path.exists(model_path):
    print("[INFO] No checkpoint found. Compiling new model...")
    model = build_model(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(learning_rate=1e-3)
    model.compile(loss="mae", optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] Loading model from {model_path} ...")
    model = load_model(model_path)

    # Update the learning rate
    print(f"[INFO] Old learning rate: {K.get_value(model.optimizer.lr)}")
    K.set_value(model.optimizer.lr, 1e-4)
    print(f"[INFO] New learning rate: {K.get_value(model.optimizer.lr)}")

# Now you can train your model with callbacks
# Example:
# model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=..., callbacks=callbacks)


[INFO] No checkpoint found. Compiling new model...


C:\Users\PMLS\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [39]:
# Convert all data to numpy arrays if they aren't already
train_X = np.array(train_X) if not isinstance(train_X, np.ndarray) else train_X
train_y = np.array(train_y) if not isinstance(train_y, np.ndarray) else train_y
validation_X = np.array(validation_X) if not isinstance(validation_X, np.ndarray) else validation_X
validation_y = np.array(validation_y) if not isinstance(validation_y, np.ndarray) else validation_y


In [43]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0401 - mae: 0.0401 - mape: 37.0203
Epoch 1: val_loss improved from inf to 0.01245, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0001-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 0.0400 - mae: 0.0400 - mape: 37.0907 - val_loss: 0.0124 - val_mae: 0.0124 - val_mape: 5.5454
Epoch 2/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0132 - mae: 0.0132 - mape: 24.8981
Epoch 2: val_loss improved from 0.01245 to 0.01067, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0132 - mae: 0.0132 - mape: 25.0311 - val_loss: 0.0107 - val_mae: 0.0107 - val_mape: 4.8765
Epoch 3/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0104 - mae: 0.0104 - mape: 41.2279
Epoch 3: val_loss improved from 0.01067 to 0.00836, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0104 - mae: 0.0104 - mape: 41.2464 - val_loss: 0.0084 - val_mae: 0.0084 - val_mape: 3.6469
Epoch 4/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0094 - mae: 0.0094 - mape: 69.7533
Epoch 4: val_loss did not improve from 0.00836
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0094 - mae: 0.0094 - mape: 69.7363 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 4.2419
Epoch 5/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0087 - mae: 0.0087 - mape: 97.9178
Epoch 5: val_loss improved from 0.00836 to 0.00789, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0087 - mae: 0.0087 - mape: 97.9307 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.7200
Epoch 6/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0083 - mae: 0.0083 - mape: 11.4756
Epoch 6: val_loss improved from 0.00789 to 0.00767, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0006-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0083 - mae: 0.0083 - mape: 11.4743 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.4159
Epoch 7/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0081 - mae: 0.0081 - mape: 126.7053
Epoch 7: val_loss did not improve from 0.00767
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0081 - mae: 0.0081 - mape: 126.5908 - val_loss: 0.0085 - val_mae: 0.0085 - val_mape: 4.0591
Epoch 8/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0077 - mae: 0.0077 - mape: 30.0943
Epoch 8: val_loss did not improve from 0.00767
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0077 - mae: 0.0077 - mape: 30.2980 - val_loss: 0.0080 - val_mae: 0.0080 - val_mape: 3.7802
Epoch 9/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0074 - mae: 0.0074 - mape: 24.2992
Epoch 9: val_loss improved from 0.00767 to 0.00739, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0009-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0074 - mae: 0.0074 - mape: 24.3822 - val_loss: 0.0074 - val_mae: 0.0074 - val_mape: 3.0153
Epoch 10/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0073 - mae: 0.0073 - mape: 18.9782
Epoch 10: val_loss improved from 0.00739 to 0.00668, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-0010-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0073 - mae: 0.0073 - mape: 18.9697 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 2.9641


In [ ]:
model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-0010-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})


y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Absolute Error (MAE): 109.46
Median Absolute Error (MedAE): 85.57
Mean Squared Error (MSE): 21134.27
Root Mean Squared Error (RMSE): 145.38
Mean Absolute Percentage Error (MAPE): 0.75 %
Median Absolute Percentage Error (MDAPE): 0.6 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
